![CH6-ADS.png](.\Media\CH6-ADS.png)

# <span style="color:#cc5500;">Working with TempDB</span>

The SQL Server system database, tempdb, has undergone a number of feature changes since SQL Server 2000.  While there are new tempdb usages and internal optimizations in SQL Server; the tempdb architecture is mostly unchanged since SQL Server 2000.  The tempdb system database is very similar to a user database.  The main difference is that data in tempdb does not persist after SQL Server shuts down.  Each time SQL Server restarts, tempdb is copied from the model database.  It inherits certain database configuration options, such as ALLOW\_SNAPSHOT\_ISOLATION, from the model database.  

Only one file group in tempdb is allowed for data and one file group for logs.  You can configure the size of the files.  When auto grow is enabled (which is the default), the file grows until the disk volume is full.  When the server restarts, the tempdb file size is reset to the configured value (the default is 8 MB).  Auto grow is temporary for tempdb (unlike other types of databases).  It is reset when SQL Server restarts.  Users can explicitly create and use tables in tempdb.  Transactions can be used to modify data in the tables.  Transactions can be rolled back.  However, there is no need to REDO them because the contents of tempdb do not persist across SQL Server restarts. 

Because the transaction log does not need to be flushed, transactions are committed faster in tempdb than in user databases.  In a user database, transactions have the ACID attributes: atomicity, concurrency, isolation, and durability.  In tempdb, transactions lose the durability attribute.  SQL Server uses tempdb to store internal objects such as the intermediate results of a query.  Most of these internal operations on tempdb do not generate log records because there is no need to roll back. So, these operations are faster.  There are some feature restrictions in tempdb.  In addition, some of the database options cannot be modified for tempdb.  Auto shrink is not allowed for tempdb.  Database shrink and file shrink capabilities are limited.  This is because many of the hidden objects that SQL Server stores in tempdb cannot be moved by shrink operations.

## <span style="color:#cc5500;">Overview</span>

The tempdb system database is a global resource that holds:

- Temporary user objects that are explicitly created.  They include global or local temporary tables and indexes, temporary stored procedures, table variables, tables returned in table-valued functions, and cursors.
- Internal objects that the database engine creates. They include:
    - Work tables to store intermediate results for spools, cursors, sorts, and temporary large object (LOB) storage.
    - Work files for hash join or hash aggregate operations.
    - Intermediate sort results for operations such as creating or rebuilding indexes (if SORT\_IN\_TEMPDB is specified), or certain GROUP BY, ORDER BY, or UNION queries. 

Each internal object uses a minimum of nine pages: an IAM page and an eight-page extent.  For more information about pages and extents, see [Pages and Extents Architecture Guide - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/relational-databases/pages-and-extents-architecture-guide?view=sql-server-ver15#pages-and-extents)  

- Version stores, which are collections of data pages that hold the data rows that support features for row versioning. There are two types: a common version store and an online-index-build version store. The version stores contain:
    - Row versions that are generated by data modification transactions in a database that uses READ COMMITTED through row versioning isolation or snapshot isolation transactions.
    - Row versions that are generated by data modification transactions for features, such as online index operations, Multiple Active Result Sets (MARS), and AFTER triggers. 

Operations within tempdb are minimally logged so that transactions can be rolled back. tempdb is re-created every time SQL Server is started so that the system always starts with a clean copy of the database. Temporary tables and stored procedures are dropped automatically on disconnect, and no connections are active when the system is shut down.  tempdb never has anything to be saved from one session of SQL Server to another. Backup and restore operations are not allowed on tempdb.

## <span style="color:#cc5500;">TempDB Space Requirement</span>

It can be difficult to estimate the tempdb space requirement for an application. This section describes the general methodology for estimating the space requirement.  These methods are not precise.  It requires experience and experiment to gain a satisfactory result.  We recommend that you always have a safety factor of about 20% more space.  Space estimates must also allow for the future growth of data in the application.

To understand the space requirement of tempdb data files, first look at which features of SQL Server use tempdb. The tempdb space requirement is different for each feature. The following features use tempdb:

- Query
- Triggers
- Snapshot isolation and read committed snapshot (RCSI)
- MARS
- Online index creation
- Temporary tables, table variables, and table-valued functions
- DBCC CHECKDB
- LOB parameters 
- Cursors
- Service Broker and event notification
- XML and LOB variable
- Query notifications
- Database mail
- Index creation
- User-defined functions

A server instance may use some or all of these features.  For some servers, one or two features might dominate tempdb space usage.  In this case, we concentrate most of the capacity planning effort on these few features.  For example, you may find out that for one server, a query requires a peak of 50 GB of tempdb space, and RCSI requires a peak of 20 GB of tempdb space, and the remaining features require only 1 GB of peak space.  It would be sufficient to allocate 85 GB (71GB + 20% overhead) of tempdb space for this server.

Applications usually send many queries to SQL Server.  If queries are sent to the server serially, the peak tempdb space requirement is determined by the query that requires most of the tempdb space.  If queries are executed at about the same time, the total tempdb space requirement is the sum of the requirement of all queries.

SQL Server has a cost-based query optimizer.  It chooses a query plan that has the lowest execution cost but it ignores the implications of this plan on tempdb.  As a result, the tempdb space requirement for a query may vary, depending on the plan. In some cases, the best plan according to the query optimizer may require more tempdb space than a less efficient plan would.  Also, when an application is upgraded, a plan change may require more space in tempdb.

For capacity planning, prepare for the worst-case scenario.  Otherwise a query may fail due to a space error in tempdb.  In some cases, auto grow can be used to expand a tempdb file until there is no space remaining in the disk volume. 

To estimate the maximum tempdb space required for a query, look at the query plan. There are several ways to view a query plan in SQL Server. SQL Server Management Studio can show the query plan that is in use, or estimate a query plan in graphical format.

## <span style="color:#cc5500;">Space required by features that use the Version Store</span>

The version store is a collection of hidden objects in tempdb that support the following features:

- Snapshot isolation
- Read committed snapshot isolation (RCSI)
- Online index build
- Trigger
- MARS

There are two version stores.  One of these stores row versions that are generated when an online index is built.  The second stores row versions that are generated for the other operations mentioned in the previous bulleted list.  New versions are inserted into the version store when there are DML operations in a database.  DML operations might be UPDATE, DELETE, and sometimes INSERT (if inserting a row with the same unique key as the row that was deleted).  Old versions are removed when they are no longer needed.  The size of the version store depends on two factors: how long the versions are needed and how many versions are generated.

One of the two version stores is for tables that are under online index build operations.  The versions in this store are needed as long as an online index build transaction is active.  For a very large index, these transactions could be active for hours or even days.  Fortunately, the versions in this store are only from the tables with active online index build operations, so this could be a very small subset of all the versions that could be generated in a large system.  For example, you are building a new nonclustered index online on a big table.  You estimate that this will take about 12 hours.  During this time, you estimate that about 600 data records in this table will be changed per hour by the concurrent DML.  The version store keeps the entire record, so you assume that the average record size of the table is 1 KB. The space required in the version store is 12\*600\*1KB = 7200KB.

The other version store is for versions generated on all tables except those that are under active online index build.  The versions in this store are needed as long as there is an active transaction that uses one of these features: snapshot isolation, read committed snapshot isolation (RCSI), triggers, or MARS.  

For example, if the longest running transaction in your SQL Server instance is a snapshot isolation transaction that is doing a complex report that takes five hours, then the versions are needed in the version store for five hours.  Different features use version stores in different ways.  Keep this in mind when estimating the number of version records generated per minute.

For snapshot isolation and RCSI, as long as the database option is set, any update in the database is versioned.  For example, the maximum update rate is 1,000 records per minute in all the databases with the options set, and the average record size is 1 KB.  The version store space required for this system with five hours of maximum transaction time is approximately 5\*60\*1000\*1KB=300MB. If a snapshot isolation transaction starts concurrently before the first transaction commits, the versions are kept until the snapshot isolation is complete.  So the version store must be kept for more than five hours.  A general method for estimating the size of version store is to multiply the size of the version store that was generated during the execution of the longest transaction by two.  When the database is enabled for snapshot isolation or RCSI, only the first update to a particular record in a transaction generates a version.

## <span style="color:#cc5500;">Space required by other features</span>

Temporary tables, table variables, and table-valued functions all use space in tempdb.  The application has explicit control over how much data goes into these objects.  The space occupied by these objects should be estimated in the same way as user tables.  LOB variables (including parameters) and XML variables all consume space in tempdb.

- DBCC CHECKDB internally starts a query to verify the consistency of data.  The execution plan of this query cannot be displayed.  However, there is an option to estimate the space requirement for this command. If you suspect that there may not be enough tempdb space, estimate the space requirement before you run the command.  Otherwise the application might fail because DBCC CHECK is competing with the application for tempdb space.
- Service Broker uses about 1 MB per dialog.  Unless the application starts a lot of dialogs without closing them, space usage should not be an issue. Also keep in mind that other features, such as event notification, use Service Broker.
- Index build has an option to sort in tempdb.  The sort requires about the same amount of tempdb space as does the index that is being built.  For online index build, we may also need a mapping index. To calculate the size of this index, multiply the average key size by the number of rows in the index. 

## <span style="color:#cc5500;">Space required for tempdb logging</span>

Most operations in tempdb are not logged.  Operations on temporary tables, table variables, table-valued functions, and user-defined tables are logged.  These are called user objects.  Sort operations in tempdb also require logging for activities related to page allocation.  The log size requirement depends on two factors: how long it is necessary to keep the log and how many log records are generated during this time.

Estimating the log space for tempdb is similar to estimating the log file size for other databases.  However, when you estimate the tempdb log size, you pay attention only to operations that generate log records.  For example, the longest running transaction in tempdb is an index build operation that uses sort.  This transaction ran for five hours. An estimated 1,000 temporary table records were fully overwritten per minute by updates. The average record size is 1 KB. The calculation for the log space required is 5\*60\*1000\*1 KB = 300 MB.  

Since log truncation can become a contention point in a heavily loaded system with a small log file, make the tempdb log file large enough to avoid truncating logs frequently. For example, if 50 MB of the log file can be generated per second in the system, the log size should be at least 500 MB so that a log truncation is performed about every 10 seconds.

## <span style="color:#cc5500;">Restrictions</span>

The following operations can't be performed on the tempdb database:

- Adding filegroups
- Backing up or restoring the database
- Changing collation. The default collation is the server collation
- Changing the database owner. tempdb is owned by sa
- Creating a database snapshot
- Dropping the database
- Dropping the guest user from the database
- Enabling Change Data Capture
- Participating in database mirroring
- Removing the primary filegroup, primary data file, or log file
- Renaming the database or primary filegroup
- Running DBCC CHECKALLOC
- Running DBCC CHECKCATALOG
- Setting the database to OFFLINE
- Setting the database or primary filegroup to READ\_ONLY

## <span style="color:#cc5500;">Optimizing tempdb performance in SQL Server</span>

The size and physical placement of the tempdb database can affect the performance of a system.  For example, if the size that's defined for tempdb is too small, part of the system-processing load might be taken up with autogrowing tempdb to the size required to support the workload every time you restart the instance of SQL Server.

- If possible, use [Database Instant File Initialization - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/relational-databases/databases/database-instant-file-initialization?view=sql-server-ver15) to improve the performance of growth operations for data files.
- Preallocate space for all tempdb files by setting the file size to a value large enough to accommodate the typical workload in the environment.  Preallocation prevents tempdb from expanding too often, which affects performance. The tempdb database should be set to autogrow to increase disk space for unplanned exceptions.
- Data files should be of equal size within each filegroup, because SQL Server uses a proportional-fill algorithm that favors allocations in files with more free space. Dividing tempdb into multiple data files of equal size provides a high degree of parallel efficiency in operations that use tempdb.
- Set the file growth increment to a reasonable size to prevent the tempdb database files from growing by too small a value. If the file growth is too small compared to the amount of data that's being written to tempdb, tempdb might have to constantly expand. That will affect performance.

To check current size and growth parameters for tempdb, use the following query:

### <span style="color:rgb(0, 204, 153);">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
SELECT name AS FileName,
    size*1.0/128 AS FileSizeInMB,
    CASE max_size
        WHEN 0 THEN 'Autogrowth is off.'
        WHEN -1 THEN 'Autogrowth is on.'
        ELSE 'Log file grows to a maximum size of 2 TB.'
    END,
    growth AS 'GrowthValue',
    'GrowthIncrement' =
        CASE
            WHEN growth = 0 THEN 'Size is fixed.'
            WHEN growth > 0 AND is_percent_growth = 0
                THEN 'Growth value is in 8-KB pages.'
            ELSE 'Growth value is a percentage.'
        END
FROM tempdb.sys.database_files;
GO

Put the tempdb database on a fast I/O subsystem. Use disk striping if there are many directly attached disks. Individual or groups of tempdb data files don't necessarily need to be on different disks or spindles unless you're also encountering I/O bottlenecks.

Put the tempdb database on disks that differ from the disks that user databases use.

Most DBA's will optimize their TempDB by spreading the files across multiple RAID disks at install time.  However, if you discover a SQL instance in your enviroment that may have tempdb located on the local C:\\ drive, or perhaps has not been modified, the following steps will guide you through the process on how to move tempdb to a more performant posture.

In [ ]:
--query the tempdb using sp_helpfile to display where your tempdb files are located, how many files you have, and display their size and growth characteristics
USE tempdb
go
EXECUTE sp_helpfile
go


## <span style="color:#cc5500;">Move TempDB</span>

One of the first steps you can take to optimize the performance of TempDB is to move TempDB to a new location (off of your C:\\ drive) to fast SAN or NAS storage.  Modify the code to a file path to an appropriate directory for your environment.  Below, I am sing K:\\  and J:\\  simply as an example

1. Step 1 is to move Tempb DB to a new location.  This step tells the master database where to create Tempdb when the SQL Service restarts
2. Step 2 is to Stop and Restart SQL Server
3. STep 3 is to alter the tempdb by creating multile files and spreading them out over multiple RAID Arrays

In [ ]:
-- Step 1. Move TempDB.  It is recommended that you only do this during a scheduled maintenance window as your SQL Server will be inaccessible for a period of time.
USE master
go
ALTER DATABASE tempdb 
MODIFY FILE 
	(name = tempdev, 
	filename = 'K:\DATA\tempdb.mdf')  -- Modify path for your environment
go
ALTER DATABASE tempdb 
MODIFY FILE 
	(name = templog, 
	filename = 'J:\LOG\templog.ldf')  -- Modify path for your environment

Step 2: 

  

- STOP and RESTART the SQL Server Service for change to take affect.  Browse to the directory that you specified in the code snippet above to verify that the new Tempdb files are there.
- Then, navigate to the original location displayed bye the query at the top and Delete the original tempdb.mdf and templog.ldf files from their original location.  They are no longer used and if you leave them there, it could cause confusion.

In [ ]:
-- Step 3
-- TempDB only supports 1 file group and you cannot add user defined filegroups.  However, you can add additional data files.  In this case TempDB database will be partitioned into 4 seperate datafiles
--Add additional files to TempDB to match the number of CPU cores.  They MUST all be configured with the same size and growth rate.

ALTER DATABASE tempdb 
ADD FILE 
	(name = tempdev2,
    filename = 'L:\DATA\tempdb2.ndf',  -- Modify path for your environment
    size = 1GB,
    maxsize = UNLIMITED,
    filegrowth = 100MB),

	(name = tempdev3,
    filename = 'M:\DATA\tempdb3.ndf',  -- Modify path for your environment
    size = 1GB,
    maxsize = UNLIMITED,
    filegrowth = 100MB),

	(name = tempdev4,
    filename = 'N:\DATA\tempdb4.ndf',  -- Modify path for your environment
    size = 1GB,
    maxsize = UNLIMITED,
    filegrowth = 100MB)

In [ ]:
--query the tempdb to see the new placement and charasteristics of your optimized TempDB

SELECT 
    name,
    file_id,
    physical_name,
    size, growth,
    state_desc
FROM sys.master_files
WHERE database_id = db_id('tempdb')


## <span style="color:#cc5500;">Additional helpful TempDB Query's</span>

Performing a simple Bing or Google search will display several links and articles on TempDB, optimizing and how to identify any contention.

Here are a couple of helpful query's for you.

In [ ]:
-- Determining the amount of free space in tempdb
SELECT SUM(unallocated_extent_page_count) AS [free pages],
  (SUM(unallocated_extent_page_count)*1.0/128) AS [free space in MB]
FROM tempdb.sys.dm_db_file_space_usage;

-- Determining the amount of space used by the version store
SELECT SUM(version_store_reserved_page_count) AS [version store pages used],
  (SUM(version_store_reserved_page_count)*1.0/128) AS [version store space in MB]
FROM tempdb.sys.dm_db_file_space_usage;

-- Determining the amount of space used by internal objects
SELECT SUM(internal_object_reserved_page_count) AS [internal object pages used],
  (SUM(internal_object_reserved_page_count)*1.0/128) AS [internal object space in MB]
FROM tempdb.sys.dm_db_file_space_usage;

-- Determining the amount of space used by user objects
SELECT SUM(user_object_reserved_page_count) AS [user object pages used],
  (SUM(user_object_reserved_page_count)*1.0/128) AS [user object space in MB]
FROM tempdb.sys.dm_db_file_space_usage;